In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Activation, LeakyReLU
from keras.layers.noise import AlphaDropout
from keras.utils.generic_utils import get_custom_objects
from keras import backend as K
from keras.optimizers import Adam, SGD
from keras.layers import GaussianNoise

def preprocess_mnist(x_train, y_train, x_test, y_test):
    x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
    x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
    input_shape = (28, 28, 1)
    
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    
    sample = GaussianNoise(0.2)
    x_train = sample(x_train/255, training=True)
    x_test = sample(x_test/255, training=True)
    
    y_train = to_categorical(y_train)
    y_test= to_categorical(y_test)
    
    return x_train, y_train, x_test, y_test, input_shape

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, y_train, x_test, y_test, input_shape = preprocess_mnist(x_train, y_train, x_test, y_test)

def build_cnn(activation,
              dropout_rate,
              optimizer):
    model = Sequential()
    
    model.add(Conv2D(32, (3, 3), activation=activation, padding='same', kernel_initializer='lecun_normal', input_shape=input_shape))
    model.add(Conv2D(32, (3, 3), activation=activation, padding='same', kernel_initializer='lecun_normal'))
    model.add(Conv2D(32, (3, 3), activation=activation, padding='same', kernel_initializer='lecun_normal'))
    model.add(Conv2D(32, (3, 3), activation=activation, padding='same', kernel_initializer='lecun_normal'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation=activation, padding='same', kernel_initializer='lecun_normal'))
    model.add(Conv2D(64, (3, 3), activation=activation, padding='same', kernel_initializer='lecun_normal'))
    model.add(Conv2D(64, (3, 3), activation=activation, padding='same', kernel_initializer='lecun_normal'))
    model.add(Conv2D(64, (3, 3), activation=activation, padding='same', kernel_initializer='lecun_normal'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation=activation, padding='same', kernel_initializer='lecun_normal'))
    model.add(Conv2D(128, (3, 3), activation=activation, padding='same', kernel_initializer='lecun_normal'))
    model.add(Conv2D(128, (3, 3), activation=activation, padding='same', kernel_initializer='lecun_normal'))
    model.add(Conv2D(128, (3, 3), activation=activation, padding='same', kernel_initializer='lecun_normal'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation=activation))
    model.add(Dense(10, activation='softmax'))
    
    model.compile(
        loss='categorical_crossentropy', 
        optimizer=optimizer, 
        metrics=['accuracy']
    )
    
    return model


def gelu(x):
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))
get_custom_objects().update({'gelu': Activation(gelu)})

def swish(x):
    return x * tf.sigmoid(x)
get_custom_objects().update({'swish': Activation(swish)})

get_custom_objects().update({'leaky-relu': Activation(LeakyReLU(alpha=0.2))})

# act_func = ['tanh', 'relu', 'leaky-relu', 'elu', 'selu', 'gelu', 'swish']
act_func = ['selu']

result = []


for activation in act_func:
    print('\nTraining with -->{0}<-- activation function\n'.format(activation))
    
    model = build_cnn(activation=activation,
                      dropout_rate=0.2,
                      optimizer=SGD())
    
    history = model.fit(x_train, y_train,
          validation_split=0.20,
          batch_size=128,
          epochs=50,
          verbose=1,
          validation_data=(x_test, y_test))
    
    result.append(history)
    
    K.clear_session()
    del model

for r in result:
    print(r.history)

11501568/11490434 [==============================] - 1s 0us/step

Training with -->selu<-- activation function

Epoch 1/50
375/375 [==============================] - 65s 56ms/step - loss: nan - accuracy: 0.1030 - val_loss: nan - val_accuracy: 0.0995
Epoch 2/50
375/375 [==============================] - 20s 52ms/step - loss: nan - accuracy: 0.0984 - val_loss: nan - val_accuracy: 0.0995
Epoch 3/50
375/375 [==============================] - 20s 52ms/step - loss: nan - accuracy: 0.0979 - val_loss: nan - val_accuracy: 0.0995
Epoch 4/50
375/375 [==============================] - 20s 53ms/step - loss: nan - accuracy: 0.1000 - val_loss: nan - val_accuracy: 0.0995
Epoch 5/50
375/375 [==============================] - 20s 54ms/step - loss: nan - accuracy: 0.0989 - val_loss: nan - val_accuracy: 0.0995
Epoch 6/50
375/375 [==============================] - 20s 53ms/step - loss: nan - accuracy: 0.1003 - val_loss: nan - val_accuracy: 0.0995
Epoch 7/50
375/375 [==============================] - 20s 53